# GREW-count (TD)

Sources : ce TD est largement repris d'une proposition de [Hee-Soo Choi](https://hschoi4.github.io/) et a été corrigé grâce à l'aide de [Bruno Guillaume](https://members.loria.fr/BGuillaume/).

La fonction request donne accès au serveur de GREW-count et permet de récupérer le nombre d’occurrences du pattern demandé. 

La fonction prend en paramètres :
- Une liste de corpus (à choisir)
- Une liste de patterns (à définir)


In [15]:
import requests
import json
from pprint import pprint

def request(corpora, patterns):
    response = requests.request(
        "POST",
        "http://count.grew.fr/count",
        headers= {'Content-Type': 'application/x-www-form-urlencoded'},
        data={ 'corpora': json.dumps(corpora), 'requests': json.dumps(patterns) }
    )
    return response.text

On définit ci-dessous les corpus à prendre en compte et les motifs (patterns) à appliquer sur ces corpus. 

Dans l'exemple, on prend en compte un corpus de l'anglais (UD_English-EWT@2.11) et on cherche tous les verbes (résultat : 28059).

In [16]:
corpus = ["UD_English-EWT@2.11" ]

patterns = {'VERB': 'pattern {V [upos=VERB]}' }

print(request(corpus, patterns))

Corpus	# sentences	VERB
UD_English-EWT@2.11	16622	28059



### Exercice 1 : déterminez si l’adjectif s’accorde en nombre avec le nom pour l'anglais, le français, l'espagnol, le japonais et le chinois.

In [68]:
corpus = [
  "UD_English-PUD@2.11",
  "UD_French-PUD@2.11",
  "UD_Spanish-PUD@2.11",
  "UD_Japanese-PUD@2.11",
  "UD_Chinese-PUD@2.11",
]

patterns = {
    'Tot' : 'pattern {N -[amod]-> A;}',
    'ADJ N' : ' pattern {N [Number]; A [Number];N -[amod]-> A;}',

    'ADJ == N' : 'pattern {N -[amod]-> A; A.Number = N.Number;}',
    'ADJ != N' : 'pattern {N -[amod]-> A; A.Number <> N.Number;}',

}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]
for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    print(f"{entete[3]} : {e[3]} ")
    if int(e[3]):
        print(f"{entete[4]} : {e[4]} ({int(e[4])/int(e[3])*100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5])/int(e[3])*100:.2f}%) ")
    else:
        print("pas de nombres")
    print()

Corpus : UD_English-PUD@2.11 
Tot : 1336 
ADJ N : 28 
ADJ == N : 23 (82.14%) 
ADJ != N : 5 (17.86%) 

Corpus : UD_French-PUD@2.11 
Tot : 1394 
ADJ N : 1390 
ADJ == N : 1385 (99.64%) 
ADJ != N : 5 (0.36%) 

Corpus : UD_Spanish-PUD@2.11 
Tot : 1311 
ADJ N : 1307 
ADJ == N : 1303 (99.69%) 
ADJ != N : 4 (0.31%) 

Corpus : UD_Japanese-PUD@2.11 
Tot : 84 
ADJ N : 0 
pas de nombres

Corpus : UD_Chinese-PUD@2.11 
Tot : 420 
ADJ N : 0 
pas de nombres



Dans les langues qui expriment les nombres des noms et adjectifs, les adjectifs et noms s'accordent dans la plupart des cas, un peu moins pour l'anglais

### Exercice 2 : pour les mêmes langues, déterminez si l’adjectif se trouve avant ou après le nom (ou les deux).

In [67]:
patterns = {
    'Tot' : 'pattern {N -[amod]-> A;}',
    'ADJ << N' : 'pattern {N -[amod]-> A; A << N;}',
    'ADJ >> N' : 'pattern {N -[amod]-> A; A >> N;}',

}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]

for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    if int(e[3]):
        print(f"{entete[3]} : {e[3]} ({int(e[3])/int(e[2])*100:.2f}%) ")
        print(f"{entete[4]} : {e[4]} ({int(e[4])/int(e[2])*100:.2f}%) ")
    else:
        print("pas de nombres")
    print()

Corpus : UD_English-PUD@2.11 
Tot : 1336 
ADJ << N : 1322 (98.95%) 
ADJ >> N : 14 (1.05%) 

Corpus : UD_French-PUD@2.11 
Tot : 1394 
ADJ << N : 435 (31.21%) 
ADJ >> N : 959 (68.79%) 

Corpus : UD_Spanish-PUD@2.11 
Tot : 1311 
ADJ << N : 399 (30.43%) 
ADJ >> N : 912 (69.57%) 

Corpus : UD_Japanese-PUD@2.11 
Tot : 84 
ADJ << N : 84 (100.00%) 
ADJ >> N : 0 (0.00%) 

Corpus : UD_Chinese-PUD@2.11 
Tot : 420 
ADJ << N : 420 (100.00%) 
ADJ >> N : 0 (0.00%) 



### Exercice 3 : choisissez 3 langues que vous ne connaissez pas et déterminez leur type (SOV, VSO, OVS, etc).

In [74]:
corpus = [
  "UD_Chinese-PUD@2.11",
  "UD_Czech-PUD@2.11",
  "UD_Swedish-PUD@2.11",
]

patterns = {
    "Tot" : """pattern {
  Verbe [upos = VERB];
  Verbe -[nsubj]-> Sujet;
  Verbe -[obj]-> Objet;
    }""",

    "SVO" : """pattern {
  Verbe [upos = VERB];
  Verbe -[nsubj]-> Sujet;
  Verbe -[obj]-> Objet;
  Sujet << Verbe;
  Verbe << Objet;
    }""",

    "SOV" : """pattern {
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Sujet << Objet;
    Objet << Verbe;
    }""",

    "VSO" : """pattern {
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Sujet;
    Sujet << Objet;
    }""",

    "VOS" : """pattern {
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Objet;
    Objet << Sujet;
    }""",

    "OSV" : """pattern {
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Objet << Sujet;
    Sujet << Verbe;
    }""",
}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]

for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    if int(e[3]):
        print(f"{entete[3]} : {e[3]} ({int(e[3])/int(e[2])*100:.2f}%) ")
        print(f"{entete[4]} : {e[4]} ({int(e[4])/int(e[2])*100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5])/int(e[2])*100:.2f}%) ")
        print(f"{entete[6]} : {e[6]} ({int(e[6])/int(e[2])*100:.2f}%) ")
        print(f"{entete[7]} : {e[7]} ({int(e[7])/int(e[2])*100:.2f}%) ")
    else:
        print("pas de nombres")
    print()

Corpus : UD_Chinese-PUD@2.11 
Tot : 656 
SVO : 656 (100.00%) 
SOV : 0 (0.00%) 
VSO : 0 (0.00%) 
VOS : 0 (0.00%) 
OSV : 0 (0.00%) 

Corpus : UD_Czech-PUD@2.11 
Tot : 395 
SVO : 280 (70.89%) 
SOV : 43 (10.89%) 
VSO : 21 (5.32%) 
VOS : 8 (2.03%) 
OSV : 20 (5.06%) 

Corpus : UD_Swedish-PUD@2.11 
Tot : 580 
SVO : 471 (81.21%) 
SOV : 0 (0.00%) 
VSO : 84 (14.48%) 
VOS : 4 (0.69%) 
OSV : 20 (3.45%) 



### Exercice 4 : pour les mêmes langues, déterminez si l'ordre sujet-verbe change si le sujet est un nom commun ou un pronom

In [76]:
patterns = {
    "Nom": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    }""",

    "SVO": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Sujet << Verbe;
    Verbe << Objet;
    }""",

    "SOV": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Sujet << Objet;
    Objet << Verbe;
    }""",

    "VSO": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Sujet;
    Sujet << Objet;
    }""",

    "VOS": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Objet;
    Objet << Sujet;
    }""",

    "OSV": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Objet << Sujet;
    Sujet << Verbe;
    }""",
}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]

for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    if int(e[3]):
        print(f"{entete[3]} : {e[3]} ({int(e[3]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[4]} : {e[4]} ({int(e[4]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[6]} : {e[6]} ({int(e[6]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[7]} : {e[7]} ({int(e[7]) / int(e[2]) * 100:.2f}%) ")
    else:
        print("pas de nombres")
    print()


Corpus : UD_Chinese-PUD@2.11 
Nom : 362 
SVO : 362 (100.00%) 
SOV : 0 (0.00%) 
VSO : 0 (0.00%) 
VOS : 0 (0.00%) 
OSV : 0 (0.00%) 

Corpus : UD_Czech-PUD@2.11 
Nom : 219 
SVO : 156 (71.23%) 
SOV : 19 (8.68%) 
VSO : 13 (5.94%) 
VOS : 4 (1.83%) 
OSV : 13 (5.94%) 

Corpus : UD_Swedish-PUD@2.11 
Nom : 215 
SVO : 176 (81.86%) 
SOV : 0 (0.00%) 
VSO : 31 (14.42%) 
VOS : 0 (0.00%) 
OSV : 8 (3.72%) 



In [78]:
patterns = {
    "Propre": """pattern {
    Sujet [upos = PROPN];
  Verbe [upos = VERB];
  Verbe -[nsubj]-> Sujet;
  Verbe -[obj]-> Objet;
    }""",

    "SVO": """pattern {
    Sujet [upos = PROPN];
  Verbe [upos = VERB];
  Verbe -[nsubj]-> Sujet;
  Verbe -[obj]-> Objet;
  Sujet << Verbe;
  Verbe << Objet;
    }""",

    "SOV": """pattern {
    Sujet [upos = PROPN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Sujet << Objet;
    Objet << Verbe;
    }""",

    "VSO": """pattern {
    Sujet [upos = PROPN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Sujet;
    Sujet << Objet;
    }""",

    "VOS": """pattern {
    Sujet [upos = PROPN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Objet;
    Objet << Sujet;
    }""",

    "OSV": """pattern {
    Sujet [upos = PROPN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Objet << Sujet;
    Sujet << Verbe;
    }""",
}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]

for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    if int(e[3]):
        print(f"{entete[3]} : {e[3]} ({int(e[3]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[4]} : {e[4]} ({int(e[4]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[6]} : {e[6]} ({int(e[6]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[7]} : {e[7]} ({int(e[7]) / int(e[2]) * 100:.2f}%) ")
    else:
        print("pas de nombres")
    print()


Corpus : UD_Chinese-PUD@2.11 
Propre : 104 
SVO : 104 (100.00%) 
SOV : 0 (0.00%) 
VSO : 0 (0.00%) 
VOS : 0 (0.00%) 
OSV : 0 (0.00%) 

Corpus : UD_Czech-PUD@2.11 
Propre : 91 
SVO : 58 (63.74%) 
SOV : 9 (9.89%) 
VSO : 8 (8.79%) 
VOS : 3 (3.30%) 
OSV : 5 (5.49%) 

Corpus : UD_Swedish-PUD@2.11 
Propre : 108 
SVO : 76 (70.37%) 
SOV : 0 (0.00%) 
VSO : 24 (22.22%) 
VOS : 4 (3.70%) 
OSV : 4 (3.70%) 



Chinois:
- Nom : 100% SVO
- Propre : 100% SVO

Tchèque :
- Nom :
    - 71,2% SVO
    - 8,7% SOV
    - 6,0% VSO
- Propre :
    - 63,7% SVO
    - 9,9% SOV
    - 8,8% VSO

Suédois:
- Nom :
    - 81.9% SVO
    - 14,4% VSO
    - 0% VOS
    - 0% OSV
- Propre :
    - 63,7% SVO
    - 22,2% VSO
    - 3,7% VOS
    - 3,7% OSV

### Exercice 5 : déterminez si le déterminant s’accorde en genre avec le nom pour l'anglais, le français, l'espagnol, le japonais et le chinois (ou d'autres langues...)

In [88]:
corpus = [
  "UD_English-PUD@2.11",
  "UD_French-PUD@2.11",
  "UD_Spanish-PUD@2.11",
  "UD_Japanese-PUD@2.11",
  "UD_Chinese-PUD@2.11",
]

patterns = {
    "Tot" : 'pattern { N [upos="NOUN"]; N -[det]-> Det;}',
    'Det N' : ' pattern {N [Gender]; Det [Gender];N [upos="NOUN"]; N -[det]-> Det;}',

    'Det == N' : 'pattern {N [upos="NOUN"]; N -[det]-> Det; Det.Gender = N.Gender;}',
    'Det != N' : 'pattern {N [upos="NOUN"]; N -[det]-> Det; Det.Gender <> N.Gender;}',
}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]
for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    print(f"{entete[3]} : {e[3]} ")
    if int(e[3]):
        print(f"{entete[4]} : {e[4]} ({int(e[4])/int(e[3])*100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5])/int(e[3])*100:.2f}%) ")
    else:
        print("pas de genres")
    print()

Corpus : UD_English-PUD@2.11 
Tot : 1779 
Det N : 0 
pas de genres

Corpus : UD_French-PUD@2.11 
Tot : 3501 
Det N : 3465 
Det == N : 3457 (99.77%) 
Det != N : 8 (0.23%) 

Corpus : UD_Spanish-PUD@2.11 
Tot : 3228 
Det N : 3216 
Det == N : 3201 (99.53%) 
Det != N : 15 (0.47%) 

Corpus : UD_Japanese-PUD@2.11 
Tot : 204 
Det N : 0 
pas de genres

Corpus : UD_Chinese-PUD@2.11 
Tot : 319 
Det N : 0 
pas de genres



### Exercice 6 : déterminez les temps verbaux des verbes (présent, passé et non indiqué)


In [90]:
patterns = {
    "Tot" : 'pattern {V [upos="VERB"];}',

    'Tense' : 'pattern {V [Tense]; V [upos="VERB"];}',

    'Pres' : 'pattern {V [upos="VERB"]; V.Tense = Pres;}',
    'Past' : 'pattern {V [upos="VERB"]; V.Tense = Past;}',
}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]

entete = x[0]
for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    print(f"{entete[3]} : {e[3]} ")
    if int(e[3]):
        print(f"{entete[4]} : {e[4]} ({int(e[4])/int(e[3])*100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5])/int(e[3])*100:.2f}%) ")
    else:
        print("pas de tense")
    print()


Corpus : UD_English-PUD@2.11 
Tot : 2156 
Tense : 1457 
Pres : 360 (24.71%) 
Past : 1097 (75.29%) 

Corpus : UD_French-PUD@2.11 
Tot : 2256 
Tense : 1834 
Pres : 714 (38.93%) 
Past : 928 (50.60%) 

Corpus : UD_Spanish-PUD@2.11 
Tot : 2189 
Tense : 1615 
Pres : 501 (31.02%) 
Past : 1081 (66.93%) 

Corpus : UD_Japanese-PUD@2.11 
Tot : 3169 
Tense : 0 
pas de tense

Corpus : UD_Chinese-PUD@2.11 
Tot : 3467 
Tense : 0 
pas de tense

